In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [0]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [0]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.model_selection import cross_val_score
import time

In [0]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_train_with_features_6.csv",
                    dtype={ \
                            'antiguedad' : np.float32, 
                            'habitaciones' : np.float32, 
                            'garages' : np.float32, 
                            'banos' : np.float32,
                            'metroscubiertos' : np.float32, 
                            'metrostotales' : np.float32, 
                            'idzona' : np.float32, 
                            'lat' : np.float32, 
                            'lng' : np.float32, 
                            'gimnasio' : np.float32, 
                            'usosmultiples' : np.float32, 
                            'piscina' : np.float32, 
                            'escuelascercanas' : np.float32,
                            'centroscomercialescercanos' : np.float32, 
                            'precio' : np.float32, 
                            'servicios_cercanos' : np.float32,
                            'utilidades_extra' : np.float32, 
                            'tipodepropiedad_mean_antiguedad' : np.float32,
                            'tipodepropiedad_mean_habitaciones ' : np.float32, 
                            'tipodepropiedad_mean_garages' : np.float32,
                            'tipodepropiedad_mean_banos' : np.float32, 
                            'tipodepropiedad_mean_metroscubiertos' : np.float32,
                            'tipodepropiedad_mean_utilidades_extra' : np.float32,
                            'tipodepropiedad_mean_servicios_cercanos' : np.float32, 
                            'ciudad_mean_antiguedad' : np.float32,
                            'ciudad_mean_habitaciones' : np.float32, 
                            'ciudad_mean_garages' : np.float32, 
                            'ciudad_mean_banos': np.float32,
                            'ciudad_mean_metroscubiertos' : np.float32, 
                            'ciudad_mean_utilidades_extra' : np.float32,
                            'ciudad_mean_servicios_cercanos' : np.float32, 
                            'provincia_mean_antiguedad' : np.float32,
                            'provincia_mean_habitaciones' : np.float32, 
                            'provincia_mean_garages' : np.float32,
                            'provincia_mean_banos' : np.float32, 
                            'provincia_mean_metroscubiertos' : np.float32,
                            'provincia_mean_utilidades_extra' : np.float32, 
                            'provincia_mean_servicios_cercanos' : np.float32,
                            'ciudad_mean_lat' : np.float32, 
                            'ciudad_mean_lng' : np.float32, 
                            'provincia_mean_lat' : np.float32,
                            'provincia_mean_lng' : np.float32, 
                            'coseno_distancia_angular_al_centro_ciudad' : np.float32,
                            'coseno_distancia_angular_al_centro_provincia' : np.float32,
                            'coseno_distancia_angular_al_centro_pais' : np.float32, 
                            'año' : np.float32, 
                            'mes' : np.float32, 
                            'dia' : np.float32,
                            'metros_no_cubiertos' : np.float32, 
                            'cantidad_espacios' : np.float32, 
                            'metros_x_espacio' : np.float32,
                            'metros_x_habitaciones' : np.float32, 
                            'metros_x_garages' : np.float32, 
                            'metros_x_banos' : np.float32,
                            'metros_x_utilidades_extra' : np.float32, 
                            'ratio_metros_cubiertos' : np.float32,
                            'distancia_minima_Terreno_comercial' : np.float32,
                            'distancia_minima_Local_Comercial' : np.float32,
                            'distancia_minima_Oficina_comercial' : np.float32,
                            'distancia_minima_Local_en_centro_comercial' : np.float32,
                            'distancia_minima_Bodega_comercial' : np.float32, 
                            'banos_preciopromedio_ciudad' : np.float32,
                            'habitaciones_preciopromedio_ciudad' : np.float32, 
                            'garages_preciopromedio_ciudad' : np.float32,
                            'banos_preciopromedio_metroscubiertos' : np.float32,
                            'habitaciones_preciopromedio_metroscubiertos' : np.float32,
                            'garages_preciopromedio_metroscubiertos' : np.float32, 
                            'precio_x_m2' : np.float32,
                            'tipodepropiedad_mean_precio' : np.float32, 
                            'titulo_cantidad_palabras_importantes' : np.float32,
                            'descripcion_cantidad_palabras_importantes' : np.float32,
                            'direccion_cantidad_palabras_importantes' : np.float32,
                            'titulo_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'descripcion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'direccion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'titulo_longitud_media_de_palabra' : np.float32,
                            'descripcion_longitud_media_de_palabra' : np.float32,
                            'direccion_longitud_media_de_palabra' : np.float32, 
                            'titulo_cantidad_stopwords' : np.float32,
                            'descripcion_cantidad_stopwords' : np.float32, 
                            'direccion_cantidad_stopwords' : np.float32,
                            'titulo_cantidad_signos_puntacion' : np.float32,
                            'descripcion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_palabras_en_mayuscula' : np.float32,
                            'direccion_cantidad_titulos' : np.float32, 
                            'titulo_cantidad_palabras_top_k' : np.float32,
                            'descripcion_cantidad_palabras_top_k' : np.float32,
                            'direccion_cantidad_palabras_top_k' : np.float32,
                            'titulo_cantidad_palabras_bottom_k' : np.float32,
                            'descripcion_cantidad_palabras_bottom_k' : np.float32,
                            'direccion_cantidad_palabras_bottom_k' : np.float32,
                            'titulo_cantidad_prefijos_top_k' : np.float32, 
                            'descripcion_cantidad_prefijos_top_k' : np.float32,
                            'direccion_cantidad_prefijos_top_k' : np.float32, 
                            'titulo_cantidad_postfijos_top_k' : np.float32,
                            'descripcion_cantidad_postfijos_top_k' : np.float32,
                            'direccion_cantidad_postfijos_top_k' : np.float32, 
                            '0' : np.float32, 
                            '1' : np.float32, 
                            '2' : np.float32, 
                            '3' : np.float32, 
                            '4' : np.float32, 
                            '5' : np.float32, 
                            '6' : np.float32,
                            '7' : np.float32,
                            'distancia_euclideana_al_origen' : np.float32, 
                            'distancia_minima_comercial' : np.float32,
                            'coseno_maximo_ciudad_pais' : np.float32,
                            'ciudad_mean_antiguedad_sobre_provincia_mean_antiguedad' : np.float32,
                            'tipodepropiead_mean_utilidades_extra_sobre_ciudad_mean_utilidades_extra' : np.float32,
                            'antiguedad_sobre_tipodepropiedad_mean_antiguedad' : np.float32,
                            'direccion_cantidad_al_menos_una_mayuscula' : np.float32,
                            'direccion_cantidad_fijos_top_k' : np.float32, 
                            'titulo_cantidad_fijos_top_k' : np.float32,
                            'titulo_palabras_top_k_sobre_total_palabras' : np.float32,
                            'ciudad_distancia_al_origen':  np.float32, 
                            'ciudad_mean_mean_todas' : np.float32,
                            'ciudad_mean_antiguedad_sobre_mean_metrocubiertos' : np.float32
                            }
                    )

In [0]:
train.set_index('id', inplace = True)

In [0]:
TARGET = 'precio'
FEATURES = ['descripcion_longitud_media_de_palabra', 'metrostotales',
       'metros_x_espacio', 'idzona', 'lat', 'titulo_longitud_media_de_palabra',
       'distancia_minima_Oficina_comercial',
       'distancia_minima_Local_en_centro_comercial',
       'distancia_minima_Bodega_comercial', 'metros_x_garages',
       'metroscubiertos', 'metros_x_habitaciones', 'metros_x_banos',
       'distancia_minima_Terreno_comercial',
       'distancia_minima_Local_Comercial',
       'titulo_cantidad_caracteres_en_palabras_importantes',
       'descripcion_cantidad_palabras_top_k',
       'descripcion_cantidad_caracteres_en_palabras_importantes',
       'coseno_distancia_angular_al_centro_provincia',
       'descripcion_cantidad_stopwords',
       'descripcion_cantidad_signos_puntacion',
       'direccion_longitud_media_de_palabra',
       'titulo_palabras_top_k_sobre_total_palabras',
       'habitaciones_preciopromedio_ciudad', 'antiguedad', 'lng',
       'garages_preciopromedio_ciudad',
       'direccion_cantidad_caracteres_en_palabras_importantes',
       'coseno_distancia_angular_al_centro_ciudad',
       'banos_preciopromedio_ciudad', 'descripcion_cantidad_prefijos_top_k',
       'precio_x_m2', 'año', 'titulo_cantidad_fijos_top_k',
       'descripcion_cantidad_palabras_importantes',
       'descripcion_cantidad_postfijos_top_k', 'ciudad_mean_mean_todas',
       'ciudad_mean_antiguedad_sobre_mean_metrocubiertos',
       'habitaciones_preciopromedio_metroscubiertos', 'cantidad_espacios',
       'direccion_cantidad_al_menos_una_mayuscula',
       'direccion_cantidad_fijos_top_k', 'ciudad_distancia_al_origen']

In [0]:
X = train[FEATURES]
y = train[TARGET]

In [0]:
K = 3
def objective(hyper_parametros):

    xgb_regressor = XGBRegressor(objective = 'reg:squarederror',**hyper_parametros)
    
    score = cross_val_score(xgb_regressor, X, y, 
                            scoring='neg_mean_absolute_error', cv=K).mean()
    
    print("SCORE: {:.3f} params {}".format(score, hyper_parametros))
    
    return {'loss': -score, 'status': STATUS_OK}

In [0]:
space = {
    "n_estimators": hp.randint("xgb_re__n_estimators", 200) + 400, 
    "learning_rate": hp.uniform("xgb_re__learning_rate", 0.011, 0.026), 
    "gamma": hp.uniform("xgb_re__gamma", 4, 5),
    "lambda": hp.uniform("xgb_re__lambda", 4, 5),
    "max_depth": hp.randint("xgb_re__max_depth", 2) + 17, 
    "min_child_weight": 9,
    'colsample_bytree': 0.8,
}

In [13]:
N_ITER = 10
rstate = np.random.RandomState(41) 
t0 = time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=N_ITER,
            rstate = rstate)
t1 = time.time()
print('Tiempo = {0:.2f} horas'.format((t1 - t0)/3600))

SCORE: -553916.542 params {'colsample_bytree': 0.8, 'gamma': 4.036816229864181, 'lambda': 4.524925387495184, 'learning_rate': 0.01985880912793387, 'max_depth': 17, 'min_child_weight': 9, 'n_estimators': 490}
SCORE: -552279.188 params {'colsample_bytree': 0.8, 'gamma': 4.731794673778704, 'lambda': 4.681262695437483, 'learning_rate': 0.025675767716743604, 'max_depth': 17, 'min_child_weight': 9, 'n_estimators': 583}
SCORE: -554297.521 params {'colsample_bytree': 0.8, 'gamma': 4.344167112503195, 'lambda': 4.282761200227482, 'learning_rate': 0.017118899298355094, 'max_depth': 17, 'min_child_weight': 9, 'n_estimators': 533}
SCORE: -554353.896 params {'colsample_bytree': 0.8, 'gamma': 4.6353020065655475, 'lambda': 4.411889456327657, 'learning_rate': 0.022347050998517123, 'max_depth': 17, 'min_child_weight': 9, 'n_estimators': 470}
SCORE: -553965.062 params {'colsample_bytree': 0.8, 'gamma': 4.6828938644391425, 'lambda': 4.964556115314149, 'learning_rate': 0.015530797581385338, 'max_depth': 17

SCORE: -553916.542 params {'colsample_bytree': 0.8, 'gamma': 4.036816229864181, 'lambda': 4.524925387495184, 'learning_rate': 0.01985880912793387, 'max_depth': 17, 'min_child_weight': 9, 'n_estimators': 490}
SCORE: -552279.188 params {'colsample_bytree': 0.8, 'gamma': 4.731794673778704, 'lambda': 4.681262695437483, 'learning_rate': 0.025675767716743604, 'max_depth': 17, 'min_child_weight': 9, 'n_estimators': 583}
SCORE: -554297.521 params {'colsample_bytree': 0.8, 'gamma': 4.344167112503195, 'lambda': 4.282761200227482, 'learning_rate': 0.017118899298355094, 'max_depth': 17, 'min_child_weight': 9, 'n_estimators': 533}
SCORE: -554353.896 params {'colsample_bytree': 0.8, 'gamma': 4.6353020065655475, 'lambda': 4.411889456327657, 'learning_rate': 0.022347050998517123, 'max_depth': 17, 'min_child_weight': 9, 'n_estimators': 470}
SCORE: -553965.062 params {'colsample_bytree': 0.8, 'gamma': 4.6828938644391425, 'lambda': 4.964556115314149, 'learning_rate': 0.015530797581385338, 'max_depth': 17, 'min_child_weight': 9, 'n_estimators': 557}
SCORE: -554364.896 params {'colsample_bytree': 0.8, 'gamma': 4.653593726834912, 'lambda': 4.0574376666821275, 'learning_rate': 0.01648418378457322, 'max_depth': 17, 'min_child_weight': 9, 'n_estimators': 545}
SCORE: -553683.875 params {'colsample_bytree': 0.8, 'gamma': 4.5840164945437785, 'lambda': 4.782100009064513, 'learning_rate': 0.021648291625051247, 'max_depth': 17, 'min_child_weight': 9, 'n_estimators': 536}
SCORE: -555338.958 params {'colsample_bytree': 0.8, 'gamma': 4.724374794921535, 'lambda': 4.290954378217474, 'learning_rate': 0.01849363432886159, 'max_depth': 18, 'min_child_weight': 9, 'n_estimators': 476}
SCORE: -555121.479 params {'colsample_bytree': 0.8, 'gamma': 4.071885411183232, 'lambda': 4.320649079078086, 'learning_rate': 0.011715818875766365, 'max_depth': 18, 'min_child_weight': 9, 'n_estimators': 590}
SCORE: -556308.062 params {'colsample_bytree': 0.8, 'gamma': 4.357572415768521, 'lambda': 4.161086352732077, 'learning_rate': 0.013465190145142723, 'max_depth': 18, 'min_child_weight': 9, 'n_estimators': 459}

SCORE: -571352.844 params {'colsample_bytree': 0.8, 'gamma': 2.128580629966828, 'lambda': 3.1207895735578397, 'learning_rate': 0.01879229950495335, 'max_depth': 19, 'min_child_weight': 233, 'n_estimators': 450}  
SCORE: -555424.953 params {'colsample_bytree': 0.8, 'gamma': 3.421288478681593, 'lambda': 2.8050062317994318, 'learning_rate': 0.017303523876455312, 'max_depth': 19, 'min_child_weight': 79, 'n_estimators': 471}  
SCORE: -619215.672 params {'colsample_bytree': 0.8, 'gamma': 2.898676314184167, 'lambda': 2.3420970476177088, 'learning_rate': 0.012190078485635922, 'max_depth': 18, 'min_child_weight': 868, 'n_estimators': 470}  
SCORE: -576189.953 params {'colsample_bytree': 0.8, 'gamma': 2.6513236925745707, 'lambda': 3.3845035395571452, 'learning_rate': 0.026588925420431023, 'max_depth': 17, 'min_child_weight': 405, 'n_estimators': 432}  
SCORE: -568047.781 params {'colsample_bytree': 0.8, 'gamma': 2.966841834728558, 'lambda': 2.8987506505065364, 'learning_rate': 0.013895214974661991, 'max_depth': 16, 'min_child_weight': 117, 'n_estimators': 460}  
SCORE: -563422.828 params {'colsample_bytree': 0.8, 'gamma': 2.9613673227767174, 'lambda': 3.373839880491179, 'learning_rate': 0.020114632696849877, 'max_depth': 16, 'min_child_weight': 152, 'n_estimators': 508}  
SCORE: -571434.266 params {'colsample_bytree': 0.8, 'gamma': 3.353521124725747, 'lambda': 3.3920245572773213, 'learning_rate': 0.016486561368219108, 'max_depth': 18, 'min_child_weight': 213, 'n_estimators': 478}  
SCORE: -583989.297 params {'colsample_bytree': 0.8, 'gamma': 2.117365120573117, 'lambda': 2.19492323122894, 'learning_rate': 0.02375923257890728, 'max_depth': 18, 'min_child_weight': 743, 'n_estimators': 591}  
SCORE: -622092.375 params {'colsample_bytree': 0.8, 'gamma': 3.3280678403107604, 'lambda': 2.8368135647372856, 'learning_rate': 0.011031410353208869, 'max_depth': 18, 'min_child_weight': 982, 'n_estimators': 531}  
SCORE: -587890.469 params {'colsample_bytree': 0.8, 'gamma': 2.625436056928425, 'lambda': 2.365395942604014, 'learning_rate': 0.020246145923565197, 'max_depth': 16, 'min_child_weight': 575, 'n_estimators': 499}  
SCORE: -571894.406 params {'colsample_bytree': 0.8, 'gamma': 3.415494410452292, 'lambda': 3.334254616899969, 'learning_rate': 0.026675993028964756, 'max_depth': 16, 'min_child_weight': 380, 'n_estimators': 526}  

SCORE: -519440.767 params {'colsample_bytree': 0.8, 'gamma': 4.7506160486034, 'lambda': 1.1183589932730134, 'learning_rate': 0.02611652524078614, 'max_depth': 18, 'min_child_weight': 9, 'n_estimators': 604, 'subsample': 0.8}  

SCORE: -578336.292 params {'gamma': 3.9592400418765896, 'lambda': 3.7750522524494228, 'learning_rate': 0.015465931263479917, 'max_depth': 18, 'min_child_weight': 344, 'n_estimators': 586} 
 